In [4]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from IPython.display import display, HTML

In [6]:
data = pd.read_csv('recipes_crop.csv')

data.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,"c(""Mix everything together and bring to a boil..."


In [7]:
columns = [
    'RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime',
    'RecipeIngredientParts', 'Calories', 'FatContent', 'SaturatedFatContent',
    'CholesterolContent', 'SodiumContent', 'CarbohydrateContent',
    'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'Keywords', 
    'Images'
]
data = data[columns]

In [8]:
def extract_quoted_strings(s):
    return re.findall(r'"([^"]*)"', s)

def clean_image_url(url_string):
    if isinstance(url_string, str):
        urls = extract_quoted_strings(url_string)
        if urls:
            return urls[0] 
    return None

data['Images'] = data['Images'].apply(clean_image_url)

In [9]:
def preprocess_data(df):
    df = df.drop_duplicates(subset=['RecipeId'])

    def convert_time(time_str):
        if pd.isna(time_str) or time_str == '': 
            return 0
        try:            
            time_str = str(time_str).replace('PT', '')
            hours = 0
            minutes = 0
            if 'H' in time_str:
                hours_part = time_str.split('H')[0]
                hours = int(hours_part) if hours_part else 0
                time_str = time_str.split('H')[1] if 'H' in time_str else time_str
            if 'M' in time_str:
                minutes_part = time_str.split('M')[0]
                minutes = int(minutes_part) if minutes_part else 0
            return hours * 60 + minutes
        except:
            return 0  

    for col in ['CookTime', 'PrepTime', 'TotalTime']:
        df[col] = df[col].apply(convert_time)

    nut_cols = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
                'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent']
    for col in nut_cols:
        df[col] = df[col] / df['RecipeServings'].replace(0, 1)  

    return df

processed_data = preprocess_data(data)

In [10]:
feature_cols = ['Calories', 'ProteinContent', 'CarbohydrateContent', 'CookTime']
features = processed_data[feature_cols].values
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

In [11]:
def prepare_training_data(features, data):
    protein_high = data['ProteinContent'] > data['ProteinContent'].median()
    carb_low = data['CarbohydrateContent'] < data['CarbohydrateContent'].median()
    y = (protein_high & carb_low).astype(int)
    return train_test_split(features, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = prepare_training_data(normalized_features, processed_data)

In [12]:
def build_model():
    model = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=4,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    return model

In [13]:
model = build_model()
model.fit(X_train, y_train)

/Users/adikazahran/anaconda3/envs/ganenv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [16:50:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [15]:
def recommend_recipes(model, features, data, calorie_target, n=3):
    scores = model.predict_proba(features)[:, 1]
    data['Score'] = scores
    mask = (data['Calories'] >= calorie_target*0.8) & (data['Calories'] <= calorie_target*1.2)
    return data[mask].nlargest(n, 'Score')

def get_user_input():
    user_data = {
        'gender': input("Jenis kelamin (Male/Female): "),
        'age': int(input("Umur (tahun): ")),
        'height': float(input("Tinggi badan (cm): ")),
        'weight': float(input("Berat badan (kg): ")),
        'activity': int(input("Tingkat aktivitas fisik (1-5): ")),
        'goal': int(input("Tujuan (1: Maintain, 2: Mild Loss, 3: Weight Loss, 4: Extreme Loss, 5: Weight Gain): "))
    }
    meals_per_day = int(input("Jumlah waktu makan per hari (3-5): "))
    return {**user_data, 'meals_per_day': meals_per_day}

In [21]:
if __name__ == "__main__":
    try:
        print("=== Food Recommendation ===")
        user = get_user_input()

        height_m = user['height']/100
        bmi = user['weight'] / (height_m ** 2)
        bmi_category = ("Underweight" if bmi < 18.5 else
                       "Normal" if bmi < 25 else
                       "Overweight" if bmi < 30 else
                       "Obese") 

        bmr = (10*user['weight'] + 6.25*user['height'] - 5*user['age'] + 
              (5 if user['gender'].lower()=='male' else -161))
        activity_multiplier = [1.2, 1.375, 1.55, 1.725, 1.9][user['activity']-1]
        maintenance = bmr * activity_multiplier

        goal_adjustment = {1:0, 2:-250, 3:-500, 4:-750, 5:300}
        target_calories = maintenance + goal_adjustment[user['goal']]

        def get_recommendations(calorie_target):
            scores = model.predict_proba(normalized_features)[:, 1]
            processed_data['Score'] = scores
            mask = (processed_data['Calories'] >= calorie_target*0.8) & \
                   (processed_data['Calories'] <= calorie_target*1.2)
            return processed_data[mask].nlargest(3, 'Score')

        breakfast = get_recommendations(target_calories*0.25)
        lunch = get_recommendations(target_calories*0.35)
        dinner = get_recommendations(target_calories*0.3)
        snack = get_recommendations(target_calories*0.1) if user['meals_per_day'] > 3 else None

        print("\n" + "="*50)
        print(f"BMI:\nNilai BMI: {bmi:.2f}\nKategori BMI: {bmi_category}")
        print(f"\nKebutuhan Kalori Harian:\nTotal Kalori: {target_calories:.0f} kkal")
        print("Pembagian Kalori:")
        print(f"Breakfast: {target_calories*0.25:.0f} kkal")
        print(f"Lunch: {target_calories*0.35:.0f} kkal")
        print(f"Dinner: {target_calories*0.3:.0f} kkal")
        if snack is not None:
            print(f"Snack: {target_calories*0.1:.0f} kkal")

        def print_meal(meal_name, meal_data):
            print(f"\n{meal_name} Menu:")
            for _, row in meal_data.iterrows():
                cuisine = re.findall(r'"([^"]*)"', str(row['Keywords']))
                cuisine = ', '.join(cuisine) if cuisine else "Unknown"
                print(f"- {row['Name']} (Cuisine: {cuisine})")
                print(f"  Kalori: {row['Calories']:.1f} kkal")
                print(f"  Protein: {row['ProteinContent']:.1f} g")
                if row['Images']:
                    display(HTML(f'<img src="{row["Images"]}" alt="Recipe Image" style="max-width: 300px; height: auto;"/><br>'))
                else:
                    display(HTML('<p><b>Gambar tidak ditemukan</b></p>'))

        print_meal("## 1. Breakfast", breakfast)
        print_meal("## 2. Lunch", lunch)
        print_meal("## 3. Dinner", dinner)
        if snack is not None:
            print_meal("## 4. Snack", snack)

    except Exception as e:
        print(f"\nError: {str(e)}")
        print("Pastikan semua input sudah benar dan coba lagi.")

=== Food Recommendation ===

BMI:
Nilai BMI: 17.30
Kategori BMI: Underweight

Kebutuhan Kalori Harian:
Total Kalori: 2488 kkal
Pembagian Kalori:
Breakfast: 622 kkal
Lunch: 871 kkal
Dinner: 746 kkal
Snack: 249 kkal

## 1. Breakfast Menu:
- Ginger Duck (Cuisine: Duck, Poultry, Meat, Very Low Carbs, Weeknight, Roast, Oven, < 4 Hours, Easy)
  Kalori: 524.8 kkal
  Protein: 15.7 g


- Lamb Grill For Two (Cuisine: Meat, < 30 Mins)
  Kalori: 728.8 kkal
  Protein: 37.4 g


- Rib-Eye Steaks With Bearnaise Butter (Cuisine: Meat, Very Low Carbs, Camping, St. Patrick's Day, < 15 Mins, Stove Top)
  Kalori: 599.0 kkal
  Protein: 30.3 g



## 2. Lunch Menu:
- Lamb Grill For Two (Cuisine: Meat, < 30 Mins)
  Kalori: 728.8 kkal
  Protein: 37.4 g


- Chicken Breast Filled With Bacon &amp; Cheese (Cuisine: Chicken, Poultry, Meat, Australian, Very Low Carbs, High Protein, High In..., Savory, < 15 Mins, Beginner Cook, Stove Top, Easy, Inexpensive)
  Kalori: 1005.1 kkal
  Protein: 80.0 g


...

- Rib-Eye Steaks With Bearnaise Butter (Cuisine: Meat, Very Low Carbs, Camping, St. Patrick's Day, < 15 Mins, Stove Top)
  Kalori: 599.0 kkal
  Protein: 30.3 g


- Taco Mac Buffalo Hot Wings (Cuisine: Poultry, Meat, < 60 Mins)
  Kalori: 610.9 kkal
  Protein: 41.8 g



## 4. Snack Menu:
- Ken's Hamburgers (Cuisine: Very Low Carbs, High Protein, Kid Friendly, High In..., Broil/Grill, < 30 Mins, Oven, Easy)
  Kalori: 273.2 kkal
  Protein: 30.4 g


- Savory Braised Short Ribs (Cuisine: Savory, Weeknight, < 4 Hours)
  Kalori: 205.0 kkal
  Protein: 7.6 g


- Blackened (cajun) Trout (Cuisine: Cajun, < 30 Mins, Easy)
  Kalori: 208.6 kkal
  Protein: 23.8 g
